In [1]:
import pandas as pd
import numpy as np
import string
import os

Tables importation and naming

In [2]:
sales = pd.read_csv("Aloe_Project_Sales.csv", sep=";")
clients = pd.read_csv("Aloe_Project_Clients.csv", sep=";")
salesreps = pd.read_csv("Aloe_Project_Sales_rep.csv", sep = ";")

Assigning the right types to the tables columns

In [3]:
#Sales
sales["code_cip_id"] = sales["code_cip_id"].astype("str")
sales["salesrep_id"] = sales["salesrep_id"].astype("str")
sales["quantity"] = sales["quantity"].astype("int")
sales["order_date"] = pd.to_datetime(sales["order_date"])

#Clients
clients["code_cip_id"] = clients["code_cip_id"].astype("str")
clients["postal_code"] = clients["postal_code"].astype("str")

#Sales_reps
salesreps["salesrep_id"] = salesreps["salesrep_id"].astype("str")

Data cleaning

In [4]:
#Removing special characters from the products' names and replacing them with the right ones
sales["article_name"] = sales["article_name"].apply(lambda x : x.replace("Ã©", "é").replace("Ãª", "ê").replace("Ã¨", "è"))

Some product's names contain what is called an EAN number at the end, it needs to be removed
All EAN numbers start with "- 8.02"

In [5]:
#Using a loop to go through the column, find the EAN number, split the string in two and keep the first part

new_values = []
for i, v in sales["article_name"].items():
    if "- 8.02" in v:
            v = v.replace("- 8.02", "removeafterthat")
            sep = "removeafterthat"
            v = v.split(sep, 1)[0]
            new_values.append(v)
    elif "- 8.02" not in v:
            v = v
            new_values.append(v)

sales["article_name"] = new_values

In [6]:
#Check
sales["article_name"].unique()

array(['Vide', 'Nutri-Pulpe 1000ml ', 'Nutri-Pulpe 500ml ',
       'Nutri-Pulpe Pêche Blanche 1000ml ',
       'Nutri-Pulpe Pêche Blanche 500ml ', 'Pur Gel Feuille 250ml ',
       'Pur Gel Flacon 100ml ', 'Aloe-Oil Huile Réparatrice 50ml ',
       "Cœur d'Aloe Cube", 'Totem Double Face',
       'Pop-Up Coeur D’Aloe Cube Offre Consommateur',
       'Présentoir de Sol Gamme Aloe',
       'Grand Aloe Vera Cartonné de Linéaire ou Vitrine',
       'Brochure Gamme Aloe Vera', 'Echantillon Nutri-Pulpe',
       'Echantillon Pur Gel',
       'Echantillon Nutri-Pulpe Goût Pêche Blanche',
       'Aloe-Oil Crème Réparatrice 150ml ',
       'Présentoir de Comptoir Aloe-Oil',
       'Réglette De Linéaire Aloe Vera',
       'Présentoir de Comptoir Gamme Aloe', "Cœur d'aloe cube",
       'Réglette de linéaire Aloe Vera', 'Totem double face',
       'Grand aloe vera cartonné de linéaire ou vitrine',
       'Pur Gel Feuille - 250ml ', 'Pur Gel Flacon - 100ml ',
       'Aloe-Oil Huile Réparatrice - 50ml 

In [7]:
#Removing spaces from the products' names
sales["article_name"] = sales["article_name"].apply(lambda x: x.strip())
#Adding caps at the begining of each word (some products are named the same but not recognised as the same because of caps changes)
sales["article_name"] = sales["article_name"].apply(lambda x: string.capwords(x, " "))

In [8]:
# Removing hyhens from products names, for example "Pur Gel Feuille - 250ml" = "Pur Gel Feuille 250ml"
without_hyphen = []
for i, v in sales["article_name"].items():
    if " - " in v:
        v = v.replace(" - ", " ")
        without_hyphen.append(v)
    elif " - " not in v:
        v = v
        without_hyphen.append(v)
    
sales["article_name"] = without_hyphen

The original excel file contains summary rows that are actually the sum of the rows below
(calculating the total revenue, and total product quantity of an order for example).They need to be removed.
The article name is always "Vide" for these rows.

There are a lot of nans in "unit_price", they are linked to sample or brochure giving.
There are also rows with 0 a "unit price", they are linked to orders with 100% discount (mainly the same situation than brochure giving)

In [9]:
#Finding and deleting the "sum rows" + finding and deleting the "Ug" rows (containing free units)
empty_names_indexes = []
Ug_indexes = []

for i,v in sales["article_name"].items():
    if "Vide" in v:
        empty_names_indexes.append(i)
    if "Ug" in v:
        Ug_indexes.append(i)
        
sales = sales.drop(empty_names_indexes)
sales = sales.drop(Ug_indexes)

In [10]:
#Finding and deleting the "NANs" in unit_price
sales = sales.dropna(subset = ["unit_price"])

In [11]:
#Finding and deleting the "0" in unit_price
zeros_in_price_indexes = [i for i,v in sales["unit_price"].items() if v == 0]
sales = sales.drop(zeros_in_price_indexes)

In [12]:
sales["unit_price"]

1        24.9
2        16.9
3        24.9
4        16.9
5        16.9
         ... 
20547     9.9
20559    24.9
20560    17.9
20561     9.9
20562     9.9
Name: unit_price, Length: 7749, dtype: float64

In [13]:
sales.describe()

,primary_key,quantity,unit_price,net_revenue
count,7749.000000,7749.000000,7749.000000,7749.000000
mean,11443.656859,11.804362,15.514221,98.299047
std,6428.607509,10.581001,4.702144,87.884123
min,81.000000,1.000000,8.900000,0.000000
25%,5777.000000,6.000000,9.900000,59.890000
50%,12102.000000,12.000000,16.900000,73.660000
75%,16975.000000,12.000000,16.900000,118.300000
max,22144.000000,150.000000,27.900000,1807.740000


CREATING A CLEAN SALES EXCEL FILE TO ANALYSE IN TABLEAU

In [14]:
sales.to_excel("cleaned_sales.xlsx", sheet_name='sales')

In this part, I will quickly start to look at the main KPIs before importing the data in tableau

Merging the clients and sales tables. Common key : code_cip_id

In [15]:
clients_sales = sales.merge(clients)
clients_sales

,primary_key,order_reference,code_cip_id,salesrep_id,article_code,article_name,quantity,unit_price,discount,net_revenue,order_date,uga,pharmacies_name,appellation_titulaire,owner_last_name,owner_first_name,commune,postal_code
0,19561,2105202234S0001,2260084,26,5046,Nutri-pulpe 1000ml,12,24.9,38%,185.26,2020-05-22,83SMA,PHARMACY NUM17193,Mme,Terence,Guilaine,COGOLIN,83310
1,19562,2105202234S0001,2260084,26,5047,Nutri-pulpe 500ml,12,16.9,38%,125.74,2020-05-22,83SMA,PHARMACY NUM17193,Mme,Terence,Guilaine,COGOLIN,83310
2,19563,2105202234S0001,2260084,26,5048,Nutri-pulpe Pêche Blanche 1000ml,6,24.9,38%,92.63,2020-05-22,83SMA,PHARMACY NUM17193,Mme,Terence,Guilaine,COGOLIN,83310
3,19564,2105202234S0001,2260084,26,5049,Nutri-pulpe Pêche Blanche 500ml,6,16.9,38%,62.87,2020-05-22,83SMA,PHARMACY NUM17193,Mme,Terence,Guilaine,COGOLIN,83310
4,19565,2105202234S0001,2260084,26,5050,Pur Gel Feuille 250ml,36,16.9,38%,377.21,2020-05-22,83SMA,PHARMACY NUM17193,Mme,Terence,Guilaine,COGOLIN,83310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,11935,1008220826S00DR,2001821,1,5052,Aloe-oil Huile Réparatrice 50ml,6,9.9,45%,32.67,2022-08-11,06MAN,PHARMACY NUM759,Mme,Melvin,Mohsen,LA NAPOULE,6210
7658,11936,1008220826S00DR,2001821,1,5053,Aloe-oil Crème Réparatrice 150ml,4,16.9,45%,37.18,2022-08-11,06MAN,PHARMACY NUM759,Mme,Melvin,Mohsen,LA NAPOULE,6210
7659,11937,1008220826S00DR,2001821,1,5599,Gyn-aloe Soin Lavant Flacon 250ml,6,12.5,45%,41.25,2022-08-11,06MAN,PHARMACY NUM759,Mme,Melvin,Mohsen,LA NAPOULE,6210
7660,11938,1008220826S00DR,2001821,1,5598,Gyn-aloe Soin Lavant Pompe 400ml,6,16.1,45%,53.13,2022-08-11,06MAN,PHARMACY NUM759,Mme,Melvin,Mohsen,LA NAPOULE,6210


Merging the sales rep and sales tables

In [16]:
#Left join because some sales reps have not sold any products yet, need to identify them
salesreps_sales = salesreps.merge(sales, how = "left")

In [17]:
salesreps_sales

,salesrep_id,last_name,first_name,primary_key,order_reference,code_cip_id,article_code,article_name,quantity,unit_price,discount,net_revenue,order_date
0,1,Martin,Marie,16110.0,1707211740S0001,2249689,5050,Pur Gel Feuille 250ml,24.0,16.9,55%,182.52,2021-07-19
1,1,Martin,Marie,16111.0,1707211740S0001,2249689,5051,Pur Gel Flacon 100ml,24.0,9.9,55%,106.92,2021-07-19
2,1,Martin,Marie,16115.0,1707211651S0001,2003434,5051,Pur Gel Flacon 100ml,12.0,9.9,45%,65.34,2021-07-19
3,1,Martin,Marie,16121.0,1707211651S0001,2003434,5597,Pure Crème 400ml,12.0,14.9,45%,98.34,2021-07-19
4,1,Martin,Marie,16122.0,1707211651S0001,2003434,5598,Gyn-aloe Soin Lavant Pompe 400ml,6.0,16.1,45%,53.13,2021-07-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7768,54,Dumont,Salomé,2037.0,2305221503S0019,2265606,5050,Pur Gel Feuille 250ml,24.0,16.9,55%,182.52,2022-05-24
7769,54,Dumont,Salomé,2038.0,2305221503S0019,2265606,5051,Pur Gel Flacon 100ml,24.0,9.9,55%,106.92,2022-05-24
7770,54,Dumont,Salomé,8508.0,2305221503S0019,2265606,5050,Pur Gel Feuille 250ml,24.0,16.9,55%,182.52,2022-05-24
7771,54,Dumont,Salomé,8509.0,2305221503S0019,2265606,5051,Pur Gel Flacon 100ml,24.0,9.9,55%,106.92,2022-05-24



SALES ANALYSIS


UGA analysis

In [18]:
#Top 10 UGA _ everything since 2020
top_10_ugas = clients_sales.groupby("uga")["net_revenue"].sum().sort_values(ascending = False).head(10)

#Reusing the series to find the top 5 products for each UGAs

In [19]:
top_10_ugas.to_frame()

,net_revenue
uga,
83DRA,67286.16
06CAG,46015.67
83SMA,20392.59
06GRA,19638.40
83FRE,18299.96
93ROS,17026.89
33BD3,16986.24
13M05,13574.93
83TL2,12816.85


In [20]:
#Top 25 products with related UGAs _ everything since 2020
clients_sales.groupby(["uga", "article_name"])["net_revenue"].sum().sort_values(ascending = False).to_frame().head(25)

net_revenue
uga   article_name                                 
83DRA Pur Gel Feuille 250ml                16392.39
      Pur Gel Flacon 100ml                 11649.57
06CAG Pur Gel Feuille 250ml                11113.47
83DRA Aloe-oil Crème Réparatrice 150ml      8383.27
33BD3 Nutri-pulpe 1000ml                    7230.96
83DRA Aloe-oil Huile Réparatrice 50ml       6726.86
06CAG Pur Gel Flacon 100ml                  6387.89
83DRA Nutri-pulpe 1000ml                    5937.19
83SMA Pur Gel Feuille 250ml                 5497.92
83FRE Pur Gel Feuille 250ml                 4972.66
33BD3 Nutri-pulpe 500ml                     4907.76
06CAG Nutri-pulpe 1000ml                    4904.81
83SAN Pur Gel Feuille 250ml                 4869.24
13M05 Pur Gel Feuille 250ml                 4585.32
06CAG Aloe-oil Crème Réparatrice 150ml      4146.88
06GRA Pur Gel Feuille 250ml                 3950.56
06CAG Aloe-oil Huile Réparatrice 50ml       3859.02
83DRA Nutri-pulpe 500ml                     3765.01
06CAG Nutri-pulpe 500ml                     3610.87
06CA2 Pur Gel Feuille 250ml                 3589.56
83TL2 Pur Gel Feuille 250ml                 3526.73
59VAL Pur Gel Feuille 250ml                 3520.58
93ROS Pur Gel Feuille 250ml                 3513.54
83SMA Pur Gel Flacon 100ml                  3377.50
93ROS Pur Gel Flacon 100ml                  3166.68

In [21]:
#Tail 10 UGA _ everything since 2020
clients_sales.groupby("uga")["net_revenue"].sum().sort_values(ascending = False).tail(10).to_frame()

,net_revenue
uga,
92CHA,332.21
37LOC,324.77
49DOU,313.85
75AUT,310.26
84CAR,292.03
84ORA,281.24
69BEL,281.24
29CON,251.48
42FEU,241.38


Products

In [22]:
#Top 10 produtcs _ everything since 2020
sales.groupby("article_name")["net_revenue"].sum().sort_values(ascending = False).head(10).to_frame()

,net_revenue
article_name,
Pur Gel Feuille 250ml,192584.9500
Pur Gel Flacon 100ml,115407.8800
Aloe-oil Crème Réparatrice 150ml,69595.6700
Aloe-oil Huile Réparatrice 50ml,63773.4425
Nutri-pulpe 1000ml,55875.5000
Nutri-pulpe 500ml,55579.7300
Nutri-pulpe Pêche Blanche 500ml,50349.3000
Nutri-pulpe Pêche Blanche 1000ml,34950.5000
Pure Crème 400ml,32213.4200


In [23]:
#Tails 10 products _ everything since 2020
sales.groupby("article_name")["net_revenue"].sum().sort_values(ascending = False).tail(25).to_frame()

,net_revenue
article_name,
Pur Gel Feuille 250ml,192584.9500
Pur Gel Flacon 100ml,115407.8800
Aloe-oil Crème Réparatrice 150ml,69595.6700
Aloe-oil Huile Réparatrice 50ml,63773.4425
Nutri-pulpe 1000ml,55875.5000
Nutri-pulpe 500ml,55579.7300
Nutri-pulpe Pêche Blanche 500ml,50349.3000
Nutri-pulpe Pêche Blanche 1000ml,34950.5000
Pure Crème 400ml,32213.4200


Sales Rep

In [24]:
#Top 10 sales reps (sellers)_ everything since 2020
salesreps_sales.groupby("first_name")["net_revenue"].sum().sort_values(ascending = False).head(10).to_frame()

,net_revenue
first_name,
Marie,179366.07
Louis,148182.61
Olivia,56585.15
Pauline,54834.82
Lara,42928.10
Bernard,42438.30
Jacques,36851.49
René,29447.93
Marcel,25453.50


In [25]:
#last 12 sellers_everything since 2020
salesreps_sales.groupby("first_name")["net_revenue"].sum().sort_values(ascending = False).tail(12).to_frame()

,net_revenue
first_name,
Christiane,0.0
Marguerite,0.0
Christine,0.0
Madeleine,0.0
Louise,0.0
Claire,0.0
Denise,0.0
Françoise,0.0
Jeanne,0.0


Buyers

In [26]:
#Top 10 buyers_ everything since 2020
clients_sales[["code_cip_id", "pharmacies_name","net_revenue"]].groupby(["code_cip_id","pharmacies_name"])["net_revenue"].sum().sort_values(ascending = False).to_frame().head(10)

,,net_revenue
code_cip_id,pharmacies_name,
2102240,PHARMACY NUM5440,16986.24
2048765,PHARMACY NUM19194,15455.49
2241522,PHARMACY NUM986,15087.60
2007231,PHARMACY NUM1764,11654.94
2235131,PHARMACY NUM19862,9530.55
2042674,PHARMACY NUM17261,9475.77
2012758,PHARMACY NUM861,9475.77
2012778,PHARMACY NUM1003,9227.67
2242919,PHARMACY NUM17204,7152.02
